In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
from tqdm import tqdm

from environment_data import Environment
from windfarm_state import Windfarm_state
from ship_plan import Ship_plan

from collections import OrderedDict

total_step_by_three_hour = int(20*365*(24/3)    /    10)      ############ とりま5年分！
total_number_of_ships = 6

/home/nama/.pyenv/versions/3.6.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


# 決め打ちするデータの読み込み
- wind_power_rank, can_work, と故障

In [2]:
def sim(actor_i):
    
    prefix = "[I actor" + str(actor_i) + "] "
    print(prefix+"read environment ...")
    print(prefix+"construct windfarm_state ...")
    print(prefix+"construct ship_plan ...")

    environment = Environment()
    windfarm_state = Windfarm_state(environment)
    ship_plan = Ship_plan(total_number_of_ships, environment, windfarm_state)
    
    need_inspection_list = []
    need_repair_list = []
    total_generated_power = []

    for t in range(total_step_by_three_hour):
        ship_plan.time_step(t)
        windfarm_state.time_step(t)

        need_inspection_list.append(sum(windfarm_state.check_need_inspection_all()))
        need_repair_list.append(sum(windfarm_state.check_need_repair_all()))
        total_generated_power.append(windfarm_state.total_calc_generated_kwh())

        need_repair_all = np.array(windfarm_state.check_need_repair_all())
        there_is_ship_all = np.array(windfarm_state.check_there_is_ship_all())
        time_from_last_inspection_all = np.array(windfarm_state.check_time_from_last_inspection_all())

        # 戦略3から引っぱってきてる
        w_repair = sum(need_repair_all & ~there_is_ship_all)/5 # max5かなっていう # どれだけ故障を許容するかが鍵？
        w_inspection = np.mean(time_from_last_inspection_all)/2160 # max2160かなっていう
        p_repair = w_repair / (w_repair + w_inspection + 1e-12)

        if t % (365*8) == 0:
            print(prefix+"t:",int(t/(365*8)), ", total_pow:", total_generated_power[-1])

    print(prefix+"total_calc_generated_kwh: {:,}".format(windfarm_state.total_calc_generated_kwh()))
    print(prefix+"total_driving_cost: {:,}".format(ship_plan.total_driving_cost))
    print(prefix+"repayment cost: {:,}".format(400000000 * total_number_of_ships))
    print(prefix+"total_profit: {:,}".format(windfarm_state.total_calc_generated_kwh() -
                          ship_plan.total_driving_cost - 400000000 *
                          total_number_of_ships))

In [3]:
N_actor = 4
import concurrent.futures
executor = concurrent.futures.ThreadPoolExecutor(max_workers=N_actor)

In [4]:
for i in range(N_actor):
    executor.submit(sim, i)

[I actor0] read environment ...
[I actor0] construct windfarm_state ...
[I actor0] construct ship_plan ...
[I actor1] read environment ...
[I actor1] construct windfarm_state ...
[I actor1] construct ship_plan ...
[I actor2] read environment ...
[I actor2] construct windfarm_state ...
[I actor2] construct ship_plan ...
[I actor3] read environment ...
[I actor3] construct windfarm_state ...
[I actor3] construct ship_plan ...
[I actor3] t: 0 , total_pow: 0
[I actor1] t: 0 , total_pow: 0
[I actor2] t: 0 , total_pow: 0
[I actor0] t: 0 , total_pow: 0
[I actor0] t: 1 , total_pow: 242067376800
[I actor2] t: 1 , total_pow: 236196756000
[I actor3] t: 1 , total_pow: 249770908800
[I actor1] t: 1 , total_pow: 235248343200
[I actor2] total_calc_generated_kwh: 469,924,912,800
[I actor2] total_driving_cost: 2,190,000,000
[I actor2] repayment cost: 2,400,000,000
[I actor2] total_profit: 465,334,912,800
[I actor0] total_calc_generated_kwh: 487,739,534,400[I actor3] total_calc_generated_kwh: 488,341,742